In [21]:
import numpy as np

### To Be Added

1. softmax forward and backward
2. convert tensor to numpy array


In [61]:
class Tensor:
    def __init__(self, data, _children=()):
        self.data = data if isinstance(data, np.ndarray) else np.array(data)
        # Set data to floats to division can be done
        self.data = self.data.astype(float) 
        self._prev = set(_children)
        self.grad = np.zeros_like(self.data)
        self.shape = self.data.shape
        self.size = self.data.size
        self._backward = lambda: None
    
    def __add__(self, other):
        # Elementwise addition. Tensors must be the same size, or one of 
        # them must be a scalar 
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data + other.data, (self, other))

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out
    
    def __sub__(self, other):    
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data - other.data, (self, other))

        def _backward():
            self.grad += out.grad
            other.grad -= out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        # Elementwise multiplication. Tensors must be the same size, or one of 
        # them must be a scalar 
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data * other.data, (self, other))

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    
    def __pow__(self, other):
        # The exponent must be a scalar
        assert isinstance(other, (int, float)), "Exponent must be a scalar (int/float)"
        out = Tensor(self.data ** other, (self, other))

        def _backward():
            self.grad += (other * self.data ** (other -1)) * out.grad
        out._backward = _backward

        return out
    
    def __matmul__(self, other):
        # Rows of self must match columns of other
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data @ other.data, (self, other))

        def _backward():
            self.grad += out.grad @ np.transpose(other.data)
            other.grad += np.transpose(out.grad @ self.data)
        out._backward = _backward

        return out
    
    def relu(self):
        out = self.data * (self.data > 0)
        out = Tensor(self.data * (self.data > 0), (self,))

        def _backward():
            self.grad += (self.data > 0) * out.grad
        out._backward = _backward

        return out
    
    def transpose(self):
        out = Tensor(np.transpose(self.data), (self,))
        
        def _backward():
            self.grad += np.transpose(out.grad)
        out._backward = _backward
        
        return out
    
    def log(self):
        out = Tensor(np.log(self.data), (self,))

        def _backward():
            self.grad += (1/self.data) * out.grad
        out._backward = _backward

        return out
    
    def exp(self):
        out = Tensor(np.exp(self.data), (self,))
        
        def _backward():
            self.grad += np.exp(self.data) * out.grad
        out._backward = _backward

        return out
    
    def sigmoid(self):
        value = 1/(1 + np.exp(-self.data))
        out = Tensor(value, (self,))
        
        def _backward():
            exp = np.exp(-self.data)
            g = exp/((1+exp)**2)
            self.grad += g * out.grad
        out._backward = _backward

        return out
    
    def tanh(self):
        val = (np.exp(self.data) - np.exp(-self.data))/(np.exp(self.data) + np.exp(-self.data))
        out = Tensor(val, (self,))

        def _backward():
            self.grad += (1 - val**2) * out.grad
        out._backward = _backward

        return out
    
    def reshape(self, *new_shape):
        old_shape = self.shape
        assert self.size == np.prod(new_shape), f'number of elements in {new_shape} must match {self.size}'
        out = Tensor(self.data.reshape(*new_shape), (self,))

        def _backward():
            self.grad += out.grad.reshape(old_shape)
        out._backward = _backward

        return out
    
    def backward(self):
        # https://github.com/karpathy/micrograd/blob/master/micrograd/engine.py
        # topological order all of the children in the graph
        
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = np.ones_like(self.data)
        for v in reversed(topo):
            v._backward() 
    
    def max(self):
        return np.max(self.data)
    
    def min(self):
        return np.min(self.data)
    
    def numpy(self):
        return np.array(self.data)
        
    @classmethod
    def zeros(cls, shape):
        assert isinstance(shape, int) or isinstance(shape, tuple), f'shape should be int or tuple insted of {type(shape)}'
        return cls(np.zeros(shape))

    @classmethod
    def ones(cls, shape):
        assert isinstance(shape, int) or isinstance(shape, tuple), f'shape should be int or tuple insted of {type(shape)}'
        return cls(np.ones(shape))
    
    @classmethod
    def normal(cls, mean=0.0, std=1.0, shape=None):
        assert isinstance(shape, int) or isinstance(shape, tuple), f'shape should be int or tuple insted of {type(shape)}'
        return cls(np.random.normal(mean, std, shape))
    
    @classmethod
    def eye(cls, N, M=None):
        return cls(np.eye(N, M))

    def __neg__(self):
        return self * -1
    
    def __radd__(self, other):
        return self + other
    
    def __rsub__(self, other):
        return other - self
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __rtruediv__(self, other):
        return other * self**-1
    
    def __repr__(self):
        return f'Tensor(data={self.data})'
    
    def __len__(self):
        return len(self.data)
    
    def __iter__(self):
        # Track the current element in the iterable
        self.current = 0
        return self
    
    def __next__(self):
        if self.current >= len(self.data):
            raise StopIteration
        current = self.data[self.current]
        self.current += 1
        return current
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value

## Test Basic Ops

In [62]:
a = Tensor([1, 2, 3])
b = Tensor([4, 5, 6])
c = a + b
print(c)
d = Tensor([5, 3.0, 2.7])
e = d/c
print(e)

Tensor(data=[5. 7. 9.])
Tensor(data=[1.         0.42857143 0.3       ])


In [63]:
a = Tensor([16.0, 2.0, 4.5])
b = Tensor([2.0, 4.0, 3.5])
c = a * b
print(c)

Tensor(data=[32.    8.   15.75])


In [64]:
a = Tensor([[2.0, 4.0, 5.0], 
           [1.0, 5.5, 2.4]])
b = 3.0
c = a ** b
print(c)

Tensor(data=[[  8.     64.    125.   ]
 [  1.    166.375  13.824]])


In [65]:
a = Tensor([[-2.0, 4.0, 5.0], 
           [1.0, -5.5, 2.4]])
a.relu()

Tensor(data=[[-0.   4.   5. ]
 [ 1.  -0.   2.4]])

In [66]:
a = Tensor([2.5])
b = Tensor([3.0])
c = a @ b
print(c)

Tensor(data=7.5)


In [67]:
a = Tensor([[-2.0, 4.0, 5.0], 
           [1.0, -5.5, 2.4]])
b = Tensor([[-2.0, 4.0], 
           [1.0, 2.4],
           [1.4, 9.0]])
c = a @ b
print(a.shape, b.shape, c.shape)

(2, 3) (3, 2) (2, 2)


In [68]:
a = Tensor([[2.0, 4.0, 5.0], 
           [1.0, 5.5, 2.4]])
a.log()

Tensor(data=[[0.69314718 1.38629436 1.60943791]
 [0.         1.70474809 0.87546874]])

In [69]:
a = Tensor([[-2.0, 4.0, 5.0], 
           [1.0, -5.5, 2.4]])
print(np.exp(a.data))
print(a.exp())

[[1.35335283e-01 5.45981500e+01 1.48413159e+02]
 [2.71828183e+00 4.08677144e-03 1.10231764e+01]]
Tensor(data=[[1.35335283e-01 5.45981500e+01 1.48413159e+02]
 [2.71828183e+00 4.08677144e-03 1.10231764e+01]])


## Test Autodiff

In [31]:
x1 = Tensor([[8.0, 2.0]])
w1 = Tensor([[3.2, 1.2]])
x1.sigmoid()
w1.tanh()
z1 = w1 * x1 - x1
q1 = z1.relu()
y1 = q1 @ x1.transpose()
p1 = y1.log()
s1 = p1.exp()
p1.backward()

print(x1.grad)
print(w1.grad)
print(z1.grad)
print(q1.grad)
print(y1.grad)
print(p1.grad)

[[0.24858757 0.00564972]]
[[0.4519774  0.02824859]]
[[0.05649718 0.01412429]]
[[0.05649718 0.01412429]]
[[0.00706215]]
[[1.]]


In [32]:
import torch

x = torch.Tensor([[8.0, 2.0]]); x.requires_grad = True
w = torch.Tensor([[3.2, 1.2]]); w.requires_grad = True
x.sigmoid()
w.tanh()
z = w * x - x; z.retain_grad()
q = z.relu(); q.retain_grad()
y = q @ x.transpose(0, 1); y.retain_grad()
p = y.log(); p.retain_grad()
s = p.exp(); s.retain_grad()
p.backward()

print(x.grad)
print(w.grad)
print(z.grad)
print(q.grad)
print(y.grad)
print(p.grad)

tensor([[0.2486, 0.0056]])
tensor([[0.4520, 0.0282]])
tensor([[0.0565, 0.0141]])
tensor([[0.0565, 0.0141]])
tensor([[0.0071]])
tensor([[1.]])


## Test Iterable

In [33]:
a = Tensor([[-2.0, 4.0, 5.0], 
           [1.0, -5.5, 2.4]])
for array in a:
    print(array)

[-2.  4.  5.]
[ 1.  -5.5  2.4]


In [34]:
a[0], a[1]

(array([-2.,  4.,  5.]), array([ 1. , -5.5,  2.4]))

In [35]:
a[0] = np.array([3.8, 9.0, 2.3])

## Make tensors

In [36]:
a = Tensor.zeros((2, 3))
a

Tensor(data=[[0. 0. 0.]
 [0. 0. 0.]])

In [37]:
b = Tensor.ones((4, 2))
b

Tensor(data=[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]])

In [38]:
c = Tensor.normal(shape=(5, 4))
c

Tensor(data=[[ 0.38252778 -0.27411117  1.50116708 -1.04678291]
 [ 1.72427404 -0.38236591  1.98548705 -0.73419999]
 [ 1.50051225  0.14430934  1.03109844 -2.02809943]
 [ 0.29127524  0.00269875  0.97721525  0.9101428 ]
 [-0.46730239  0.45090472 -0.08714041  0.07924003]])

In [39]:
d = Tensor.eye(6)
d

Tensor(data=[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]])

In [40]:
e = Tensor.normal(shape=(4, 5))
print(e)
f = e.numpy()
print(f)

Tensor(data=[[-2.29175332 -1.62432208 -1.94227389 -0.04600475  0.03931947]
 [-0.62092735 -1.68806266  0.14150988 -0.19513705 -0.18440252]
 [-1.09380258 -0.27197402 -0.02712507  1.29861898 -0.35948258]
 [-1.7850084   0.2943795  -0.76582953  0.27154558 -0.01450988]])
[[-2.29175332 -1.62432208 -1.94227389 -0.04600475  0.03931947]
 [-0.62092735 -1.68806266  0.14150988 -0.19513705 -0.18440252]
 [-1.09380258 -0.27197402 -0.02712507  1.29861898 -0.35948258]
 [-1.7850084   0.2943795  -0.76582953  0.27154558 -0.01450988]]
